In [1]:
import os 
import datetime
from pathlib import Path, PurePath, PosixPath
from dotenv import load_dotenv, find_dotenv

import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
from skimage.transform import resize

import os 
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
sys.path.append(os.getcwd())
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2

from src.preprocessing.split.train_test_split import train_test_split
from sklearn.model_selection import GroupShuffleSplit, StratifiedKFold

In [2]:
# Load labels
DATASET_FOLDER = Path(os.environ.get('CHEXPERT_DEV_DATASET_DIRECTORY'))
data = pd.read_csv(DATASET_FOLDER / 'train.csv', index_col=[0])
data = data.fillna(0.0)
data = data

In [12]:
labels = data.columns[5:19].to_list()

In [16]:
train, test = train_test_split(data, test_size=0.2, labels=labels)

In [17]:
train_size = train.size / data.size
test_size = test.size / data.size
print('Train split size: ', train_size)
print('Test split size: ', test_size)
print(train.shape)
print(test.shape)

Train split size:  0.8251569104649673
Test split size:  0.17484308953503266
(12884, 20)
(2730, 20)


In [28]:

d = {'Pathology': [], 'Positive %': [], 'Uncertain %': [], 'Negative %': []}
for label in labels:
    values = train.groupby(label)
    d['Pathology'].append(label)

    positive = values.size()[1.0] if 1.0 in values.size() else 0
    positive_percent = positive / train.shape[0] * 100
    d['Positive %'].append(round(positive_percent))

    uncertain = values.size()[-1.0] if -1.0 in values.size() else 0
    uncertain_percent = uncertain / train.shape[0] * 100
    d['Uncertain %'].append(round(uncertain_percent))

    negative = values.size()[-0.0] if -0.0 in values.size() else 0
    negative_percent = negative / train.shape[0] * 100
    d['Negative %'].append(round(negative_percent))

d_val = {'Pathology': [], 'Positive %': [], 'Uncertain %': [], 'Negative %': []}
for label in labels:
    values = test.groupby(label)
    d_val['Pathology'].append(label)

    positive = values.size()[1.0] if 1.0 in values.size() else 0
    positive_percent = positive / test.shape[0] * 100
    d_val['Positive %'].append(round(positive_percent))

    uncertain = values.size()[-1.0] if -1.0 in values.size() else 0
    uncertain_percent = uncertain / test.shape[0] * 100
    d_val['Uncertain %'].append(round(uncertain_percent))

    negative = values.size()[-0.0] if -0.0 in values.size() else 0
    negative_percent = negative / test.shape[0] * 100
    d_val['Negative %'].append(round(negative_percent))

In [29]:
df = pd.DataFrame(d)
df = df.set_index('Pathology')

df_val = pd.DataFrame(d_val)
df

,Positive %,Uncertain %,Negative %
Pathology,,,
No Finding,9.0,0.0,91.0
Enlarged Cardiomediastinum,5.0,6.0,89.0
Cardiomegaly,14.0,3.0,83.0
Lung Opacity,49.0,2.0,49.0
Lung Lesion,4.0,1.0,95.0
Edema,24.0,6.0,70.0
Consolidation,7.0,12.0,81.0
Pneumonia,3.0,9.0,89.0
Atelectasis,15.0,14.0,70.0


In [30]:
df_val

,Pathology,Positive %,Uncertain %,Negative %
0,No Finding,15.0,0.0,85.0
1,Enlarged Cardiomediastinum,5.0,6.0,89.0
2,Cardiomegaly,11.0,4.0,85.0
3,Lung Opacity,44.0,3.0,53.0
4,Lung Lesion,4.0,1.0,95.0
5,Edema,22.0,6.0,72.0
6,Consolidation,6.0,12.0,81.0
7,Pneumonia,3.0,9.0,88.0
8,Atelectasis,16.0,17.0,68.0
9,Pneumothorax,7.0,2.0,91.0
